## RECOMMEDATION SYSTEM
Data Description:

Unique ID of each anime.
Anime title.
Anime broadcast type, such as TV, OVA, etc.
anime genre.
The number of episodes of each anime.
The average rating for each anime compared to the number of users who gave ratings.


Number of community members for each anime.
Objective:
The objective of this assignment is to implement a recommendation system using cosine similarity on an anime dataset. 
Dataset:
Use the Anime Dataset which contains information about various anime, including their titles, genres,No.of episodes and user ratings etc.


Tasks:

Data Preprocessing:

Load the dataset into a suitable data structure (e.g., pandas DataFrame).
Handle missing values, if any.
Explore the dataset to understand its structure and attributes.


In [20]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Akshata\OneDrive\Desktop\Assignment\Recommendation System\anime.csv")
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  11954 non-null  float64
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 672.5+ KB


In [5]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [31]:
# Handling missing values and fixing the 'episodes' column

# Replace 'Unknown' or other non-numeric values in 'episodes' with NaN, then convert to numeric
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

# Fill missing values for 'genre' and 'type' with 'Unknown', and for 'rating' with the mean rating
df['genre']=df['genre'].fillna('Unknown')
df['type']=df['type'].fillna('Unknown')
df['rating']=df['rating'].fillna(df['rating'].mean())

# Checking the updated dataset info and first few rows to confirm changes
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  11954 non-null  float64
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 672.5+ KB


(None,
    anime_id                              name  \
 0     32281                    Kimi no Na wa.   
 1      5114  Fullmetal Alchemist: Brotherhood   
 2     28977                          Gintama°   
 3      9253                       Steins;Gate   
 4      9969                     Gintama&#039;   
 
                                                genre   type  episodes  rating  \
 0               Drama, Romance, School, Supernatural  Movie       1.0    9.37   
 1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV      64.0    9.26   
 2  Action, Comedy, Historical, Parody, Samurai, S...     TV      51.0    9.25   
 3                                   Sci-Fi, Thriller     TV      24.0    9.17   
 4  Action, Comedy, Historical, Parody, Samurai, S...     TV      51.0    9.16   
 
    members  
 0   200630  
 1   793665  
 2   114262  
 3   673572  
 4   151266  )

In [33]:
# Summary of missing values after handling
print("\nMissing values after handling:")
print(df.isnull().sum())


Missing values after handling:
anime_id      0
name          0
genre         0
type          0
episodes    340
rating        0
members       0
dtype: int64


In [35]:
# Exploring the dataset structure - statistical summary
print("\nStatistical summary of the dataset:")
print(df.describe(include='all'))


Statistical summary of the dataset:
            anime_id                     name   genre   type      episodes  \
count   12294.000000                    12294   12294  12294  11954.000000   
unique           NaN                    12292    3265      7           NaN   
top              NaN  Shi Wan Ge Leng Xiaohua  Hentai     TV           NaN   
freq             NaN                        2     823   3787           NaN   
mean    14058.221653                      NaN     NaN    NaN     12.382550   
std     11455.294701                      NaN     NaN    NaN     46.865352   
min         1.000000                      NaN     NaN    NaN      1.000000   
25%      3484.250000                      NaN     NaN    NaN      1.000000   
50%     10260.500000                      NaN     NaN    NaN      2.000000   
75%     24794.500000                      NaN     NaN    NaN     12.000000   
max     34527.000000                      NaN     NaN    NaN   1818.000000   

              rating      

In [37]:
df.isnull().sum()

anime_id      0
name          0
genre         0
type          0
episodes    340
rating        0
members       0
dtype: int64

In [41]:
df.dtypes

anime_id      int64
name         object
genre        object
type         object
episodes    float64
rating      float64
members       int64
dtype: object

Feature Extraction:

Decide on the features that will be used for computing similarity (e.g., genres, user ratings).
Convert categorical features into numerical representations if necessary.
Normalize numerical features if required.


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Step 1: Feature Extraction

# Using 'genre' and 'type' as categorical features
# Combine 'genre' and 'type' for text vectorization
df['combined_features'] = df['genre'] + ' ' + df['type']

# Step 2: Convert Categorical Features into Numerical Representations
# Initialize TfidfVectorizer for text features
tfidf = TfidfVectorizer(stop_words='english')

# Apply TF-IDF on the combined features
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

# Step 3: Normalize Numerical Features (e.g., 'rating' and 'episodes')
# Replace NaN values in 'episodes' with the mean before normalization
df['episodes'] = df['episodes'].fillna(df['episodes'].mean())

# Select numerical columns for normalization
numerical_features = df[['rating', 'episodes']]

# Normalize numerical features using MinMaxScaler
scaler = MinMaxScaler()
normalized_numerical_features = scaler.fit_transform(numerical_features)

# Step 4: Combine All Features (TF-IDF + Normalized Numerical Features)
# Convert the TF-IDF matrix to a DataFrame and append the normalized numerical features
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=df.index)
normalized_numerical_df = pd.DataFrame(normalized_numerical_features, columns=['rating_norm', 'episodes_norm'])

# Concatenate the TF-IDF features and normalized numerical features
final_features = pd.concat([tfidf_df, normalized_numerical_df], axis=1)

# Display the combined feature set
print("Final Feature Set for Cosine Similarity:")
print(final_features.head())


Final Feature Set for Cosine Similarity:
          0         1    2    3    4         5    6    7         8    9  ...  \
0  0.000000  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0  0.406427  0.0  ...   
1  0.285137  0.307353  0.0  0.0  0.0  0.000000  0.0  0.0  0.324992  0.0  ...   
2  0.244698  0.000000  0.0  0.0  0.0  0.196013  0.0  0.0  0.000000  0.0  ...   
3  0.000000  0.000000  0.0  0.0  0.0  0.000000  0.0  0.0  0.000000  0.0  ...   
4  0.244698  0.000000  0.0  0.0  0.0  0.196013  0.0  0.0  0.000000  0.0  ...   

    44        45        46        47   48   49   50   51  rating_norm  \
0  0.0  0.502594  0.000000  0.000000  0.0  0.0  0.0  0.0     0.924370   
1  0.0  0.000000  0.000000  0.252040  0.0  0.0  0.0  0.0     0.911164   
2  0.0  0.000000  0.000000  0.216295  0.0  0.0  0.0  0.0     0.909964   
3  0.0  0.000000  0.797356  0.292296  0.0  0.0  0.0  0.0     0.900360   
4  0.0  0.000000  0.000000  0.216295  0.0  0.0  0.0  0.0     0.899160   

   episodes_norm  
0       0.000000  
1

Recommendation System:

Design a function to recommend anime based on cosine similarity.
Given a target anime, recommend a list of similar anime based on cosine similarity scores.
Experiment with different threshold values for similarity scores to adjust the recommendation list size.

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity matrix using the final feature set
cosine_sim = cosine_similarity(final_features)

# Function to recommend anime based on cosine similarity
def recommend_anime(target_title, threshold=0.1, num_recommendations=10):
    # Get the index of the target anime
    try:
        idx = df[df['name'] == target_title].index[0]
    except IndexError:
        return f"Anime titled '{target_title}' not found in the dataset."

    # Get the pairwise similarity scores for the target anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Filter recommendations based on the threshold
    filtered_scores = [score for score in sim_scores if score[1] > threshold]

    # Get the indices of the most similar anime
    anime_indices = [i[0] for i in filtered_scores[1:num_recommendations + 1]]  # Exclude itself and limit number

    # Return the titles of the recommended anime
    recommended_titles = df['name'].iloc[anime_indices].tolist()

    return recommended_titles

# Example: Recommend anime similar to 'Kimi no Na wa.' with a threshold of 0.2
recommended_anime = recommend_anime('Kimi no Na wa.', threshold=0.2, num_recommendations=10)
print("Recommended Anime:")
print(recommended_anime)


Recommended Anime:
['Aura: Maryuuin Kouga Saigo no Tatakai', 'Harmonie', 'Kokoro ga Sakebitagatterunda.', 'Air Movie', 'Wind: A Breath of Heart OVA', 'Wind: A Breath of Heart (TV)', 'Suki ni Naru Sono Shunkan wo.: Kokuhaku Jikkou Iinkai', 'Clannad Movie', 'Shakugan no Shana II (Second)', 'Shakugan no Shana']


Evaluation:

Split the dataset into training and testing sets.
Evaluate the recommendation system using appropriate metrics such as precision, recall, and F1-score.
Analyze the performance of the recommendation system and identify areas of improvement.

In [58]:
from sklearn.model_selection import train_test_split

# Assuming anime_df is your DataFrame containing anime data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Reset the index
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)


In [60]:
def generate_recommendations_for_test_set(df, threshold=0.1, num_recommendations=10):
    recommendations = {}
    for title in df['name']:
        recs = recommend_anime(title, threshold=threshold, num_recommendations=num_recommendations)
        recommendations[title] = recs
    return recommendations

# Generate recommendations for the test set
test_recommendations = generate_recommendations_for_test_set(test_df)


In [61]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to calculate precision, recall, and F1-score
def evaluate_recommendations(df, recommendations):
    y_true = []
    y_pred = []

    for title, recs in recommendations.items():
        # True positives (anime in the test set that are also recommended)
        true_anime = set(test_df[test_df['name'] == title]['name'].tolist())
        for rec in recs:
            if rec in true_anime:
                y_true.append(1)  # It's a relevant recommendation
            else:
                y_true.append(0)  # Not a relevant recommendation
            
            # All recommended anime are considered as predictions
            y_pred.append(1 if rec in true_anime else 0)

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    return precision, recall, f1

# Evaluate the recommendations
precision, recall, f1 = evaluate_recommendations(df, test_recommendations)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")


Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000


Identify Areas of Improvement
Threshold Adjustment: Experiment with different thresholds to see how they affect precision and recall.

Feature Engineering: Consider adding more features or refining existing ones to improve the accuracy of the cosine similarity calculation.

 Alternative Algorithms: Explore other recommendation algorithms, such as collaborative filtering or content-based filtering, and compare their performance against the cosine similarity-based method.
 
 User Feedback: If possible, collect user feedback on the recommendations to fine-tune the system further.

Interview Questions:
1. Can you explain the difference between user-based and item-based collaborative filtering?

User Based: Here, we look for the users who have rated various items in the same way and then find the rating of the missing item with the help of these users.
Item Based: Here, we explore the relationship between the pair of items (the user who bought Y, also bought Z). We find the missing rating with the help of the ratings given to the other items by the user.


2. What is collaborative filtering, and how does it work?

Collaborative filtering is a popular technique used in recommendation systems to suggest items (such as movies, books, products, etc.) to users based on the preferences and behaviors of other users. The underlying assumption of collaborative filtering is that users who have agreed in the past will agree in the future, making it possible to recommend items that users with similar tastes have liked or rated positively.

